### Assignment : Week 1 
## Modeling simple RL problems by making their MDPs in Python

We will create the MDPs for some of the example problems from Grokking textbook. For the simple environments, we can just hardcode the MDPs into a dictionary by exhaustively encoding the whole state space and the transition function. We will also go through a more complicated example where the state space is too large to be manually coded and we need to implement the transition function based on some state parameters.

Later on, you will not need to implement the MDPs of common RL problems yourself, most of the work is already done by the OpenAI Gym library, which includes models for most of the famous RL envis.

You can start this assignment during/after reading Grokking Ch-2.

## Environment 0 - Bandit Walk

Let us consider the BW environment on Page 39. 

State Space has 3 elements, states 0, 1 and 2.
States 0 and 2 are terminal states and state 1 is the starting state.

Action space has 2 elements, left and right.

The environment is deterministic - transition probability of any action is 1.

Only 1 (State, Action, State') tuple has positive reward, (1, Right, 2) gives the agent +1 reward.

We'll model this MDP as a dictionary. This code is an example for the upcoming exercises.

In [1]:
bw_mdp = {

    0 : {
        "Right" : [(1, 0, 0, True)],
        "Left" : [(1, 0, 0, True)]
    },

    1 : {
        "Right" : [(1, 2, 1, True)],
        "Left" : [(1, 0, 0, True)]
    },

    2 : {
        "Right" : [(1, 2, 0, True)],
        "Left" : [(1, 2, 0, True)]
    }
    
}

Note that by convention, all actions from terminal states still lead to the same state with reward 0.

## Environment 1 - Slippery Walk

Let us now look at the BSW environment on Page 40. We'll model a slightly modified version of BSW with 7 states instead (i.e the SWF envi on Page 67). It will be useful in the coming weeks.

Here, states 0 and 6 are terminal states and state 3 is the starting state.

Action space has again 2 elements, left and right.

The environment is now stochastic, transition probability of any action is as follows -
If agent chooses `Right` at a non-terminal state,
- $50\%$ times it will go to `Right` state
- $33\frac{1}{3} \%$ times it will stay in same state
- $16\frac{2}{3}\%$ times it will go to `Left`state

This time, 2 different (State, Action, State') tuples have positive rewards, you need to find them.

We'll again model this MDP as a dictionary. Part of the code is written for you.

In [5]:
swf_mdp = {

    0 : {
        "Right" : [(1, 0, 0, True)],
        "Left" : [(1, 0, 0, True)],
    },

    1 : {
        "Right" : [
            (1/2, 2, 0, False),
            (1/3, 1, 0, False),
            (1/6, 0, 0, True),
        ],
        "Left" : [
            (1/2, 0, 0, True),
            (1/3, 1, 0, False),
            (1/6, 2, 0, False),
        ],
    }, 

    # to be added 
    2 : {
        "Right" : [
            (1/2, 3, 0, False),
            (1/3, 2, 0, False),
            (1/6, 1, 0, False),
        ],
        "Left" : [
            (1/2, 1, 0, False),
            (1/3, 2, 0, False),
            (1/6, 3, 0, False),
        ],
    },
    3 : {
        "Right" : [
            (1/2, 4, 0, False),
            (1/3, 3, 0, False),
            (1/6, 2, 0, False),
        ],
        "Left" : [
            (1/2, 2, 0, False),
            (1/3, 3, 0, False),
            (1/6, 4, 0, False),
        ],
    },
    4 : {
        "Right" : [
            (1/2, 5, 0, False),
            (1/3, 4, 0, False),
            (1/6, 3, 0, False),
        ],
        "Left" : [
            (1/2, 3, 0, False),
            (1/3, 4, 0, False),
            (1/6, 5, 0, False),
        ],
    },
    5 : {
        "Right" : [
            (1/2, 6, 1, True),
            (1/3, 5, 0, False),
            (1/6, 4, 0, False),
        ],
        "Left" : [
            (1/2, 4, 0, False),
            (1/3, 5, 0, False),
            (1/6, 6, 1, True),
        ],
    },
    6 : {
        "Right" : [(1, 6, 0, True)],
        "Left" : [(1, 6, 0, True)],
    },
    
}

Feel free to automate filling this MDP, but ensure that it is correctly filled as it'll be back in next week's assignment.

## Environment 2 - Frozen Lake Environment

This environment is described on Page 46.

The FL environment has a large state space, so it's better to generate most of the MDP via Python instead of typing stuff manually.

Note that all 5 states - 5, 7, 11, 12, 15 are terminal states, so keep that in mind while constructing the MDP.

There are 4 actions now - Up, Down, Left, Right.

The environment is stochastic, and states at the border of lake will require separate treatment.



Yet again we will model this MDP as a (large) dictionary.

In [9]:
from pprint import pprint

# Define the dimensions of the grid
GRID_SIZE = 4
TERMINAL_STATES = {5, 7, 11, 12, 15}
ACTIONS = ["Up", "Down", "Left", "Right"]

# Define the transitions for stochastic behavior
PROBABILITIES = {
    "Intended": 1/3,
    "Left": 1/3,
    "Right": 1/3
}

def left_action(action):
    if(action=="Up"):
        return "Left"
    elif(action=="Left"):
        return "Down"
    elif(action=="Down"):
        return "Right"
    elif(action=="Right"):
        return "Up"
    
def right_action(action):
    if(action=="Left"):
        return "Up"
    elif(action=="Down"):
        return "Left"
    elif(action=="Right"):
        return "Down"
    elif(action=="Up"):
        return "Right" 

def get_next_state(state, action):
    """Determines the next state based on the current state and action."""
    row, col = divmod(state, GRID_SIZE)
    
    
    if action == "Up":
        row = max(row - 1, 0)
    elif action == "Down":
        row = min(row + 1, GRID_SIZE - 1)
    elif action == "Left":
        col = max(col - 1, 0)
    elif action == "Right":
        col = min(col + 1, GRID_SIZE - 1)

    return row * GRID_SIZE + col

In [10]:
def generate_transitions(state):
    """Generates transitions for a given state."""
    if state in TERMINAL_STATES:
        return {action: [(1.0, state, 0, True)] for action in ACTIONS}  # Stays in the same state

    transitions = {}
    for action in ACTIONS:
        intended_state = get_next_state(state, action)
        left_state = get_next_state(state, left_action(action))
        right_state = get_next_state(state, right_action(action))

        transitions[action] = [
            (PROBABILITIES["Intended"], intended_state, 0, intended_state in TERMINAL_STATES),
            (PROBABILITIES["Left"], left_state, 0, left_state in TERMINAL_STATES),
            (PROBABILITIES["Right"], right_state, 0, right_state in TERMINAL_STATES)
        ]

    return transitions

You might need to do some stuff manually, but make sure to automate most of it.

You can check your implementation of the FL environment by comparing it with the one in OpenAI Gym.

You don't need to worry about Gym right now, we'll set it up in the coming weeks. But here is the code to import an MDP.

In [13]:
swf_mdp = {}
for state in range(GRID_SIZE * GRID_SIZE):
    swf_mdp[state] = generate_transitions(state)

Since the imported MDP is also just a dictionary, we can just print it.

In [15]:
pprint(swf_mdp)

{0: {'Down': [(0.3333333333333333, 4, 0, False),
              (0.3333333333333333, 1, 0, False),
              (0.3333333333333333, 0, 0, False)],
     'Left': [(0.3333333333333333, 0, 0, False),
              (0.3333333333333333, 4, 0, False),
              (0.3333333333333333, 0, 0, False)],
     'Right': [(0.3333333333333333, 1, 0, False),
               (0.3333333333333333, 0, 0, False),
               (0.3333333333333333, 4, 0, False)],
     'Up': [(0.3333333333333333, 0, 0, False),
            (0.3333333333333333, 0, 0, False),
            (0.3333333333333333, 1, 0, False)]},
 1: {'Down': [(0.3333333333333333, 5, 0, True),
              (0.3333333333333333, 2, 0, False),
              (0.3333333333333333, 0, 0, False)],
     'Left': [(0.3333333333333333, 0, 0, False),
              (0.3333333333333333, 5, 0, True),
              (0.3333333333333333, 1, 0, False)],
     'Right': [(0.3333333333333333, 2, 0, False),
               (0.3333333333333333, 1, 0, False),
               (